In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load path

In [2]:
## Load Path
root_path = 'drive/MyDrive/Skip-GNN/'

data_path = 'drive/MyDrive/Skip-GNN/data/'
edges_path = data_path + 'edges/'
iav_root_path = data_path + 'IAV/'

# Import Libaries

In [3]:
from __future__ import division
from __future__ import print_function

In [4]:
import time

import pandas as pd
import numpy as np

import scipy.sparse as sp
from scipy import sparse

# import tensorflow as tf
import tensorflow.compat.v1 as tf

# import pickle as pkl
# import networkx as nx
from sklearn.preprocessing import scale, normalize

import sys, getopt

flags = tf.app.flags
FLAGS = flags.FLAGS

In [5]:
### initialization.py ###

def weight_variable_glorot(input_dim, output_dim, name=""):
    
    """
      Create a weight variable with Glorot & Bengio (AISTATS 2010) initialization.
    """
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = tf.random_uniform([input_dim, output_dim], minval=-init_range,
                                maxval=init_range, dtype=tf.float64)
    return tf.Variable(initial, name=name)

# load_data

## protein embedding

In [6]:
def load_protein_embedding(p_mtd):
  
  ## ../data/IAV/protein_embeddings/(p_mtd).npz

  read_p_emb = sparse.load_npz(iav_root_path + 'protein_embeddings/' + p_mtd + '.npz')
  allx = read_p_emb.toarray()
  
  norm_features = normalize(allx) ## from sklearn.preprocessing (default l2 normalization)
  
  print("==================== Features ====================")
  print(norm_features)

  return norm_features

In [7]:
# load_protein_embedding('CT')

In [8]:
def load_edgelist():
  edgelist = pd.read_csv(iav_root_path + 'idx_gae_edgelist.txt', sep=' ', header=None)
  edgelist.columns=['Protein1_ID', 'Protein2_ID','label']

  pos_edgelist = edgelist[edgelist['label'] == 1].drop_duplicates().reset_index(drop=True)
  neg_edgelist = edgelist[edgelist['label'] == 0].drop_duplicates().reset_index(drop=True)
  
  # print(edgelist)

  return pos_edgelist, neg_edgelist

In [9]:
def load_data():
  
  '''
        Load protein_list
  '''
  protein_list_df = pd.read_csv(iav_root_path + 'protein_list.csv')

  '''
    features: Codings for proteins
    Call load_protein_embedding()
  '''
  features = load_protein_embedding('CT')
  
  '''
    Call load_edgelist()
  '''
  posEdges, negEdges = load_edgelist()
  
  # print(posEdges)
  # print(negEdges)

  for i in range(len(posEdges)):
    x = posEdges['Protein1_ID'][i]
    y = posEdges['Protein2_ID'][i]

  '''
    Positive edges
  '''
  ## Generate adjacency matrix
  num_node = features.shape[0]
  adj = np.zeros((num_node,num_node))

  for i in range(len(posEdges)):
    x = posEdges['Protein1_ID'][i]
    y = posEdges['Protein2_ID'][i]

    adj[x][y] = 1
    adj[y][x] = 1

  adj = sp.csr_matrix(adj) ## Compressed Sparse Row Format

  print("==================== Adjacency Matrix ====================")
  print(adj)

  '''
    Negative edges
  '''
  ## Generate highly negative sets
  rows = []
  cols = []
  
  for i in range(len(negEdges)):
    rows.append(int(negEdges['Protein1_ID'][i]))
    cols.append(int(negEdges['Protein2_ID'][i]))
  
  X = np.array(rows)
  Y = np.array(cols)
  falseEdges = np.vstack((X,Y)).transpose()

  print("==================== False Edges ====================")
  print(falseEdges)
  
  return adj, features, falseEdges

In [10]:
# adj, features, falseEdges = load_data()

# preprocessing.py

In [11]:
# preprocess_graph, construct_feed_dict, sparse_to_tuple, construct_optimizer_list

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    return sparse_to_tuple(adj_normalized)

def construct_feed_dict(adj_normalized, adj, features, placeholders):
    # construct feed dictionary
    feed_dict = dict()
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['adj']: adj_normalized})
    feed_dict.update({placeholders['adj_orig']: adj})
    return feed_dict

def construct_optimizer_list(num_node, posEdges, falseEdges):
    
    mask_index = []
    
    for x in posEdges:
        temp = x[0] * num_node + x[1]
        mask_index.append(temp)

    for x in falseEdges:
        temp = x[0] * num_node + x[1]
        mask_index.append(temp)

    return np.array(mask_index)

## make_test_edges

In [12]:
def make_test_edges(weightRate, adj, falseEdges):
    '''
      Function to build test set with 10% positive links and 10% highly negative links
      NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
    '''

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()

    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]

    posNum = edges.shape[0]
    num_test = int(np.floor(posNum / 10.))
    num_train = edges.shape[0] - num_test

    # Generate positive sets
    all_edge_idx = list(range(posNum))
    np.random.shuffle(all_edge_idx)
    train_edge_idx = all_edge_idx[:num_train]
    train_edges = edges[train_edge_idx]
    test_edge_idx = all_edge_idx[num_train:(num_train + num_test)]
    test_edges = edges[test_edge_idx]
    
    falseNum = falseEdges.shape[0]
    num_neg_test = int(np.floor(falseNum/10.))
    num_neg_train = falseNum - num_neg_test

    # Generate negative sets
    all_edge_neg_idx = list(range(falseNum))
    np.random.shuffle(all_edge_neg_idx)
    train_edge_neg_idx = all_edge_neg_idx[:num_neg_train]
    test_edge_neg_idx = all_edge_neg_idx[num_neg_train:(num_neg_train+num_neg_test)]
    train_edges_false = falseEdges[train_edge_neg_idx]
    test_edges_false = falseEdges[test_edge_neg_idx]

    # Re-build training adjacency matrix
    adj_train = np.zeros(adj.shape)
    facNeg = -0.001
    facPos = weightRate * facNeg

    for x in train_edges:
        adj_train[x[0]][x[1]] = facPos
        adj_train[x[1]][x[0]] = facPos
    
    for x in train_edges_false:
        adj_train[x[0]][x[1]] = facNeg
        adj_train[x[1]][x[0]] = facNeg

    adj_train = sp.csr_matrix(adj_train)


    return adj_train, train_edges, train_edges_false, test_edges, test_edges_false

# layers.py

In [13]:
# from initialization import *

# Global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}

def get_layer_uid(layer_name=''):
    """
      Helper function, assigns unique layer IDs
    """
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def dropout_sparse(x, keep_prob, num_nonzero_elems):
    """
      Dropout for sparse tensors. Currently fails for very large sparse tensors (>1M elements)
    """
    noise_shape = [num_nonzero_elems]
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor), dtype=tf.bool)
    pre_out = tf.sparse_retain(x, dropout_mask)
    #return pre_out * (1./keep_prob)
    return x


class Layer(object):
    
    """
      Base layer class. Defines basic API for all layer objects.

      # Properties
          name: String, defines the variable scope of the layer.

      # Methods
          _call(inputs): Defines computation graph of layer
              (i.e. takes input, returns output)
          __call__(inputs): Wrapper for _call()
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.issparse = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            outputs = self._call(inputs)
            return outputs


class GraphConvolution(Layer):
    """
      Basic graph convolution layer for undirected graph without edge labels.
    """
    def __init__(self, input_dim, output_dim, adj, dropout=0., act=tf.nn.relu, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)
        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = weight_variable_glorot(input_dim, output_dim, name="weights")
        self.dropout = dropout
        self.adj = adj
        self.act = act

    def _call(self, inputs):
        x = inputs
        #x = tf.nn.dropout(x, 1-self.dropout)
        x = tf.matmul(x, self.vars['weights'])
        x = tf.sparse_tensor_dense_matmul(self.adj, x)
        outputs = self.act(x)
        return outputs


class GraphConvolutionSparse(Layer):
    """
      Graph convolution layer for sparse inputs.
    """
    def __init__(self, input_dim, output_dim, adj, features_nonzero, dropout=0., act=tf.nn.relu, **kwargs):
        super(GraphConvolutionSparse, self).__init__(**kwargs)
        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = weight_variable_glorot(input_dim, output_dim, name="weights")
        self.dropout = dropout
        self.adj = adj
        self.act = act
        self.issparse = True
        self.features_nonzero = features_nonzero

    def _call(self, inputs):
        x = inputs
        x = dropout_sparse(x, 1-self.dropout, self.features_nonzero)
        x = tf.sparse_tensor_dense_matmul(x, self.vars['weights'])
        x = tf.sparse_tensor_dense_matmul(self.adj, x)
        outputs = self.act(x)
        return outputs


class InnerProductDecoder(Layer):
    """
      Decoder model layer for link prediction.
    """
    def __init__(self, input_dim, dropout=0., act=tf.nn.sigmoid, **kwargs):
        super(InnerProductDecoder, self).__init__(**kwargs)
        self.dropout = dropout
        self.act = act

    def _call(self, inputs):
        inputs = tf.nn.dropout(inputs, 1-self.dropout)
        x = tf.transpose(inputs)
        x = tf.matmul(inputs, x)
        x = tf.reshape(x, [-1])
        outputs = self.act(x)
        return 

# model.py

In [14]:
# from layers import GraphConvolution, GraphConvolutionSparse, InnerProductDecoder

class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg

        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

    def fit(self):
        pass

    def predict(self):
        pass


class GCNModelAE(Model):
    def __init__(self, placeholders, num_features, features_nonzero, **kwargs):
        super(GCNModelAE, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = num_features
        self.features_nonzero = features_nonzero
        self.adj = placeholders['adj']
        self.dropout = placeholders['dropout']
        self.build()

    def _build(self):
        self.hidden1 = GraphConvolutionSparse(input_dim=self.input_dim,
                                              output_dim=FLAGS.hidden1,
                                              adj=self.adj,
                                              features_nonzero=self.features_nonzero,
                                              act=tf.nn.relu,
                                              dropout=self.dropout,
                                              logging=self.logging)(self.inputs)

        self.embeddings = GraphConvolution(input_dim=FLAGS.hidden1,
                                           output_dim=FLAGS.hidden2,
                                           adj=self.adj,
                                           act=lambda x: x,
                                           dropout=self.dropout,
                                           logging=self.logging)(self.hidden1)

        self.z_mean = self.embeddings

        self.reconstructions = InnerProductDecoder(input_dim=FLAGS.hidden2,
                                        act=lambda x: x,
                                      logging=self.logging)(self.embeddings)


class GCNModelVAE(Model):
    def __init__(self, placeholders, num_features, num_nodes, features_nonzero, **kwargs):
        super(GCNModelVAE, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = num_features
        self.features_nonzero = features_nonzero
        self.n_samples = num_nodes
        self.adj = placeholders['adj']
        self.dropout = placeholders['dropout']
        self.build()

    def _build(self):
        self.hidden1 = GraphConvolutionSparse(input_dim=self.input_dim,
                                              output_dim=FLAGS.hidden1,
                                              adj=self.adj,
                                              features_nonzero=self.features_nonzero,
                                              act=tf.nn.relu,
                                              dropout=self.dropout,
                                              logging=self.logging)(self.inputs)

        self.z_mean = GraphConvolution(input_dim=FLAGS.hidden1,
                                       output_dim=FLAGS.hidden2,
                                       adj=self.adj,
                                       act=lambda x: x,
                                       dropout=self.dropout,
                                       logging=self.logging)(self.hidden1)

        self.z_log_std = GraphConvolution(input_dim=FLAGS.hidden1,
                                          output_dim=FLAGS.hidden2,
                                          adj=self.adj,
                                          act=lambda x: x,
                                          dropout=self.dropout,
                                          logging=self.logging)(self.hidden1)

        self.z = self.z_mean + tf.random_normal([self.n_samples, FLAGS.hidden2], dtype=tf.float64) * tf.exp(self.z_log_std)

        self.reconstructions = InnerProductDecoder(input_dim=FLAGS.hidden2,
                                        act=lambda x: x,
                                      logging=self.logging)(self.z)

# optimizer.py

In [15]:
class OptimizerAE(object):
    def __init__(self, preds, labels, pos_weight, norm, learning_rate):
        preds_sub = preds
        labels_sub = labels

        self.cost = norm * tf.reduce_mean(
            tf.nn.weighted_cross_entropy_with_logits(logits=preds_sub, targets=labels_sub, pos_weight=pos_weight))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  # Adam Optimizer

        self.opt_op = self.optimizer.minimize(self.cost)
        self.grads_vars = self.optimizer.compute_gradients(self.cost)

        self.correct_prediction = tf.equal(tf.cast(tf.greater_equal(tf.sigmoid(preds_sub), 0.5), tf.int32),
                                           tf.cast(labels_sub, tf.int32))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

class OptimizerVAE(object):
    def __init__(self, preds, labels, model, num_nodes, pos_weight, norm, learning_rate):
        preds_sub = preds
        labels_sub = labels

        self.cost = norm * tf.reduce_mean(
            tf.nn.weighted_cross_entropy_with_logits(logits=preds_sub, targets=labels_sub, pos_weight=pos_weight))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)  # Adam Optimizer

        # Latent loss
        self.log_lik = self.cost
        self.kl = (0.5 / num_nodes) * tf.reduce_mean(tf.reduce_sum(1 + 2 * model.z_log_std - tf.square(model.z_mean) -
                                                                   tf.square(tf.exp(model.z_log_std)), 1))
        self.cost -= self.kl

        self.opt_op = self.optimizer.minimize(self.cost)
        self.grads_vars = self.optimizer.compute_gradients(self.cost)

        self.correct_prediction = tf.equal(tf.cast(tf.greater_equal(tf.sigmoid(preds_sub), 0.5), tf.int32),
                                           tf.cast(labels_sub, tf.int32))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

#trainGcn

In [16]:
# from optimizer import OptimizerAE, OptimizerVAE
# from model import GCNModelAE, GCNModelVAE
# from preprocessing import preprocess_graph, construct_feed_dict, sparse_to_tuple, construct_optimizer_list

## from load_data(): features
## from make_test_edges() adj_train, train_edges, train_edges_false, test_edges, test_edges_false

def train_gcn(features, adj_train, train_edges, train_edges_false, test_edges, test_edges_false):
    
    ## Settings [__init__ (self, args)]

    flags = tf.app.flags
    FLAGS = flags.FLAGS

    flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
    flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
    flags.DEFINE_integer('hidden1', 32, 'Number of units in hidden layer 1.') ## To be optimized
    flags.DEFINE_integer('hidden2', 16, 'Number of units in hidden layer 2.')
    flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
    flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
    
    flags.DEFINE_string('model', 'gcn_vae', 'Model string.')
    flags.DEFINE_integer('features', 1, 'Whether to use features (1) or not (0).')

    model_str = FLAGS.model

    # 1-dim index array, used in cost function to only focus on those interactions with high confidence
    mask_index = construct_optimizer_list(features.shape[0], train_edges, train_edges_false)

    # Store original adjacency matrix (without diagonal entries) for later
    adj_orig = adj_train
    adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
    adj_orig.eliminate_zeros()

    adj = adj_train

    if FLAGS.features == 0:
        features = sp.identity(features.shape[0])  # featureless

    # Some preprocessing
    adj_norm = preprocess_graph(adj)

    # Define placeholders
    placeholders = {
        'features': tf.sparse_placeholder(tf.float64),
        'adj': tf.sparse_placeholder(tf.float64),
        'adj_orig': tf.sparse_placeholder(tf.float64),
        'dropout': tf.placeholder_with_default(0., shape=())
    }

    num_nodes = adj.shape[0]

    features = sparse_to_tuple(features.tocoo()) ## Coordinate Format (COO) - fast format for constructing sparse matrices
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]

    # Create model
    model = None
    if model_str == 'gcn_ae':
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif model_str == 'gcn_vae':
        model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)

    pos_weight = 1
    norm = 1

    ## VAE
    # pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
    # norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)


    # Optimizer
    with tf.name_scope('optimizer'):
        if model_str == 'gcn_ae':
            opt = OptimizerAE(preds=model.reconstructions,
                          labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                      validate_indices=False), [-1]),
                          pos_weight=pos_weight,
                          norm=norm,
                          mask=mask_index)
        elif model_str == 'gcn_vae':
            opt = OptimizerVAE(preds=model.reconstructions,
                           labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                       validate_indices=False), [-1]),
                           model=model, num_nodes=num_nodes,
                           pos_weight=pos_weight,
                           norm=norm,
                           mask=mask_index)

    # Initialize session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    adj_label = adj_train + sp.eye(adj_train.shape[0])
    adj_label = sparse_to_tuple(adj_label)

    # Train model
    for epoch in range(FLAGS.epochs):

        t = time.time()
        
        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_label, features, placeholders)
        feed_dict.update({placeholders['dropout']: FLAGS.dropout})
        
        # Run single weight update
        outs = sess.run([opt.opt_op, opt.cost], feed_dict=feed_dict)
        
        # Compute average loss
        avg_cost = outs[1]
        avg_accuracy = outs[2]

        print("Epoch:", '%04d' % (epoch + 1),
              "train_loss=", "{:.5f}".format(avg_cost),
              "train_acc=", "{:.5f}".format(avg_accuracy),
              "time=", "{:.5f}".format(time.time() - t))

        # print("Epoch:", '%04d' % (epoch+1), "train_loss=", "{:.5f}".format(outs[1]))


    print("Optimization Finished!")
    
    ## Return embedding for each protein
    emb = sess.run(model.z_mean,feed_dict=feed_dict) ## save_embeddings(self, output, node_list)
    
    print("==================== Embedding ====================")
    print(emb)
    print(emb.shape)

    return emb

## Run program here

In [17]:
# from preprocessing import make_test_edges
# from input_data import load_data
# from trainGcn import train_gcn
# from trainNN import generate_data, train_nn

def run_program(weightRate): ## default weightRate = 1
    
    adj, features, falseEdges = load_data()

    # Generate training and test data 
    adj_train, train_edges, train_edges_false, test_edges, test_edges_false = make_test_edges(weightRate, adj, falseEdges)

    print(adj_train.shape)
    print(train_edges.shape, train_edges_false.shape)
    print(test_edges.shape, test_edges_false.shape)

    # Embeddings returned by S-VGAE
    emb = train_gcn(features,adj_train,train_edges,train_edges_false,test_edges,test_edges_false)

    print(emb)

    # # Read FNN train, test, val datasets
    # X_train,Y_train = generate_data(emb, train_edges, train_edges_false)
    # X_test,Y_test = generate_data(emb, test_edges, test_edges_false)

    # # Final softmax classifier
    # acc = train_nn(X_train,Y_train,X_test,Y_test)
    # print 'accuracy:',acc[0]
    # print 'sensitivity:',acc[1]
    # print 'specificity:',acc[2]
    # print 'precision:',acc[3]

In [18]:
### Call train()
run_program(1)

==================== Features ====================
[[0.11339099 0.09719228 0.14578842 ... 0.         0.         0.        ]
 [0.04855793 0.08092989 0.09711586 ... 0.         0.         0.        ]
 [0.         0.05268837 0.05268837 ... 0.         0.         0.        ]
 ...
 [0.15762208 0.23643312 0.07881104 ... 0.         0.         0.        ]
 [0.         0.17052613 0.05684204 ... 0.         0.         0.        ]
 [0.         0.11639519 0.08729639 ... 0.         0.         0.        ]]
==================== Adjacency Matrix ====================
  (0, 41)	1.0
  (0, 43)	1.0
  (0, 46)	1.0
  (0, 51)	1.0
  (0, 53)	1.0
  (0, 54)	1.0
  (0, 60)	1.0
  (0, 67)	1.0
  (0, 74)	1.0
  (0, 76)	1.0
  (0, 84)	1.0
  (0, 90)	1.0
  (0, 93)	1.0
  (0, 97)	1.0
  (0, 99)	1.0
  (0, 105)	1.0
  (0, 112)	1.0
  (0, 116)	1.0
  (0, 119)	1.0
  (0, 124)	1.0
  (0, 126)	1.0
  (0, 133)	1.0
  (0, 136)	1.0
  (0, 137)	1.0
  (0, 142)	1.0
  :	:
  (15677, 1)	1.0
  (15677, 2)	1.0
  (15677, 7)	1.0
  (15677, 9)	1.0
  (15678, 0)

UnrecognizedFlagError: ignored

In [ ]:
# def main():
    
#     opts, args = getopt.getopt(sys.argv[1:],"d:w:",["dataset=", "wr="])
    
#     dataset = "IAV" ## Placeholder
#     weightRate = 1

#     for opt, arg in opts:
#         if opt == '--dataset':
#             dataset = arg
#         if opt == '--wr':
#             weightRate=int(arg)

#     # print(dataset)
#     # print(weightRate)

#     ### Call train()
#     train(weightRate)

In [ ]:
# train(1) 

In [ ]:
## SAVE EMBEDDING
# np.save('../data/IAV/SVGAE_embeddings/' + emb_name + '.npy', emb) ## save path